In [1]:
from collections import defaultdict
import gensim
import nltk
import pandas as pd
import re
prog = re.compile('\.[A-Z]')
Head_check = re.compile('^[A-Z][^A-Z]+$')
from spacy.lang.en import English
nlp = English()

/opt/conda/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/train.csv")
all_document = train_df["comment_text"]+test_df["comment_text"]
# kaggle だとメモリ足らない
del train_df, test_df

In [3]:
# .が繋がっているとわかれないので
def splitter(text, split_words, include = True):
    split_words = [i.span()[0] for i in split_words]
    word_list = []
    word = ""
    for i in range(len(text)):
        word += text[i]
        if i in split_words:
            word_list.append(word_compile(word[:-1]))
            if include:
                word_list.append(".")
            word = ""
    word_list.append(word_compile(word))
    return word_list
splitter("borrow.Incoming", prog.finditer("borrow.Incoming"))

['borrow', '.', 'Incoming']

In [4]:
# 先頭大文字への対処．固有名詞には基本触れない
lemmatizer = nltk.stem.WordNetLemmatizer()
def word_compile(word, lemmatize = True):
    # 先頭のみ大文字
    check = Head_check.match(word)
    if check is not None:
        word = word[0].lower() + word[1:]
    # 見出し語化
    if lemmatize:
        word = lemmatizer.lemmatize(word)
    return word
print(word_compile("They"))
print(word_compile("TRUMP"))
print(word_compile("WordNet"))

they
TRUMP
WordNet


In [ ]:
# sentence.split() はnltk.word_torknize(sentence) でも可
# I'll や don't などの対応が変わる
sentences = [[word_compile(word) if len(prog.findall(word)) == 0 else splitter(word, prog.finditer(word))[0] 
              for word in nltk.word_tokenize(sentence)] for sentence in all_document]
sentences

In [6]:
# 出現回数の少ない順にソート（確認のため）
word_count = defaultdict(lambda: 0)
for sentence in sentences:
    for word in sentence:
        word_count[word] += 1
word_count = dict(sorted(word_count.items(), key = lambda kv: kv[1]))
word_count

{'losers.haha': 1,
 'comment.ur': 1,
 'hahahahahahahahhha': 1,
 'it.hahahahahahahahhha': 1,
 'FFFFUUUUUUUUUUUUUUUFFFFUUUUUUUUUUUUUUU': 1,
 'basis.disqus': 1,
 'Belden': 1,
 'CALIFORNIA.let': 1,
 '//yeezy-season2.com/Loving': 1,
 '//yeezy-season2.com/': 1,
 'lawrence.and': 1,
 '//yardpdx.com/leasing/Took': 1,
 '//yardpdx.com/leasing/': 1,
 '🍕🍕🍕Have': 1,
 '🍕🍕🍕': 1,
 'v=c15hy8dXSpsThe': 1,
 '//youtu.be/T424sWq1SkEWhat': 1,
 'agendadefending': 1,
 'genocide.you': 1,
 'PCC.': 1,
 '//newpittsburghcourieronline.com/2013/07/10/more-african-americans-get-involved-in-anthrocon-every-year/It': 1,
 '//newpittsburghcourieronline.com/2013/07/10/more-african-americans-get-involved-in-anthrocon-every-year/': 1,
 'eatsOne': 1,
 'Iannarone': 1,
 'listenersOf': 1,
 '//multco.us/elections/november-4-2008-measure-no-26-96Here': 1,
 '//multco.us/elections/november-4-2008-measure-no-26-96': 1,
 '//www.theportlanddream.comVoodoo': 1,
 '//www.theportlanddream.com': 1,
 'nimbys': 1,
 'www.businessinsider.com/it

In [7]:
# MIN_COUNT に到達しない単語にも情報があれば付与したり，UNDEFINED などにしたほうが後々楽
MIN_COUNT = 10

In [ ]:
word2vec = gensim.models.Word2Vec(sentences, window=5, min_count=MIN_COUNT , iter=5, negative=5)

In [ ]:
word2vec.wv.most_similar("suck")

In [ ]:
# Bidirectional LSTM network
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out